In [65]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from scipy import stats


### Data Preparation

In [66]:
train=pd.read_csv("C:/Users/karth/Documents/Kaggle/Purchase Prediction/train.csv")
test=pd.read_csv("C:/Users/karth/Documents/Kaggle/Purchase Prediction/test_v2.csv")
print(train.shape)
print(test.shape)

frames = [train, test]
my_data = pd.concat(frames,keys=['train', 'test'])
print(my_data.shape)


my_data['location'] = my_data['location'].fillna(0)
my_data['C_previous'] = my_data['C_previous'].fillna(99)
my_data['duration_previous'] = my_data['duration_previous'].fillna(99)
my_data['car_value'] = my_data['car_value'].fillna(99)

my_data= my_data.dropna(axis='columns')
my_data.isnull().sum(axis = 0)

(665249, 25)
(198856, 25)
(864105, 25)


customer_ID          0
shopping_pt          0
record_type          0
day                  0
time                 0
state                0
location             0
group_size           0
homeowner            0
car_age              0
car_value            0
age_oldest           0
age_youngest         0
married_couple       0
C_previous           0
duration_previous    0
A                    0
B                    0
C                    0
D                    0
E                    0
F                    0
G                    0
cost                 0
dtype: int64

In [67]:
def one_hot(df, cols):
    for col in cols:
        df[col] = df[col].astype('category')
    df=pd.concat([df,pd.get_dummies(df[cols])], axis=1)
    df.drop(cols, axis=1, inplace=True)
    return df
        
def time_convert(df):
    df['time'] = df['time'].apply(lambda x: (int(x[:2]) * 60) + int(x[3:]))
    df['time_sin'] = np.sin((df['time'] * np.pi) / 1800)
    df['time_cos'] = np.cos((df['time'] * np.pi) / 1800)
    df = df.drop(['time'], axis=1)
    return df

cat_cols = ['shopping_pt','day', 'state',
       'group_size', 'homeowner', 'car_age','car_value', 'age_oldest', 'age_youngest',
       'married_couple','C_previous','duration_previous']
my_data = one_hot(my_data,cat_cols)
my_data = time_convert(my_data)

#Split data back in to train & Test

my_train=my_data.loc['train']
my_test=my_data.loc['test']
print(my_train.shape)
print(my_test.shape)

(665249, 295)
(198856, 295)


In [68]:
#Getting the purchased Data as columns in my train
purchased_data=my_train.loc[my_train.groupby('customer_ID').record_type.idxmax()][['customer_ID','A','B','C','D','E','F','G']]
purchased_data=purchased_data.rename(columns={"A": "A_purchased","B": "B_purchased","C": "C_purchased","D": "D_purchased","E": "E_purchased","F": "F_purchased","G": "G_purchased",})
my_train=my_train.merge(purchased_data, left_on='customer_ID', right_on='customer_ID', how='left')

In [69]:
# #One hot encoding 
# options = ['A','B','C','D','E','F','G']
# my_train = one_hot(my_train,options)
# my_test = one_hot(my_test,options)


In [70]:
my_train.head()

customer_ID  record_type  location  A  B  C  D  E  F  G  ...  \
0     10000000            0   10001.0  1  0  2  2  1  2  2  ...   
1     10000000            0   10001.0  1  0  2  2  1  2  1  ...   
2     10000000            0   10001.0  1  0  2  2  1  2  1  ...   
3     10000000            0   10001.0  1  0  2  2  1  2  1  ...   
4     10000000            0   10001.0  1  0  2  2  1  2  1  ...   

   duration_previous_99.0  time_sin  time_cos  A_purchased  B_purchased  \
0                       0  0.782608  0.622515            1            0   
1                       0  0.785857  0.618408            1            0   
2                       0  0.785857  0.618408            1            0   
3                       0  0.786935  0.617036            1            0   
4                       0  0.948324  0.317305            1            0   

   C_purchased  D_purchased  E_purchased  F_purchased  G_purchased  
0            2            2            1            2            1  
1            2            2            1            2            1  
2            2            2            1            2            1  
3            2            2            1            2            1  
4            2            2            1            2            1  

[5 rows x 302 columns]

### Prediction of G

In [71]:
#splitting my train data in to train & validation
y = my_train['G_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

Training data size (532199, 293)
validation data size (133050, 293)


In [72]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)
y_pred_prob_val=logreg.predict_proba(X_val) 
metrics.confusion_matrix(y_val,y_pred_val)

array([[19742,  7062,  1236,    22],
       [ 3554, 42882,  4059,    54],
       [ 1627,  8612, 32701,    27],
       [  405,  2095,  8921,    51]], dtype=int64)

In [73]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

             precision    recall  f1-score   support

          1       0.78      0.70      0.74     28062
          2       0.71      0.85      0.77     50549
          3       0.70      0.76      0.73     42967
          4       0.33      0.00      0.01     11472

avg / total       0.69      0.72      0.68    133050

0.7168432919954905


In [74]:
#Applying the model on test Data
G_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_G=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
G_pred['G_predicted']=logreg.predict(my_test_G)
counts=G_pred.groupby('customer_ID').count()
counts=counts.rename({'G_predicted': 'counts'}, axis=1) 
G_pred=G_pred.merge(counts, left_on='customer_ID', right_index=True, how='left')
# G_predicted_mode=G_pred.groupby('customer_ID')['G_predicted'].apply(lambda x: x.mode().iloc[0])
G_predicted_mode=G_pred.groupby('customer_ID')['G_predicted'].last()

G_predicted_mode=G_predicted_mode.to_frame()
G_predicted_mode.reset_index(level=0, inplace=True)

### Prediction of C & D

In [75]:
#MODEL FOR C

#splitting my train data in to train & validation
y = my_train['C_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 293)
validation data size (133050, 293)
             precision    recall  f1-score   support

          1       0.78      0.85      0.81     40253
          2       0.72      0.55      0.62     28031
          3       0.80      0.83      0.82     51874
          4       0.71      0.78      0.74     12892

avg / total       0.77      0.77      0.77    133050

0.7720405862457723


In [76]:
#Applying the model on test Data
C_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_C=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
C_pred['C_predicted']=logreg.predict(my_test_C)

In [77]:

C_predicted_mode=C_pred.groupby('customer_ID')['C_predicted'].last()
# C_predicted_mode=C_pred.groupby('customer_ID')['C_predicted'].apply(lambda x: x.mode().iloc[0])
C_predicted_mode=C_predicted_mode.to_frame()
C_predicted_mode.reset_index(level=0, inplace=True)

In [78]:
#MODEL FOR D

#splitting my train data in to train & validation
y = my_train['D_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 294)
validation data size (133050, 294)
             precision    recall  f1-score   support

          1       0.72      0.78      0.75     17192
          2       0.75      0.51      0.61     30016
          3       0.88      0.96      0.92     85842

avg / total       0.83      0.84      0.83    133050

0.8384066140548666


In [79]:
#Applying the model on test Data
my_test_D=my_test.copy()
D_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_D=my_test_D.merge(C_predicted_mode, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_D=my_test_D.drop(['record_type','customer_ID'], axis=1)

In [80]:
logreg = LogisticRegression()
logreg.fit(X, y)
D_pred['D_predicted']=logreg.predict(my_test_D)

D_predicted_mode=D_pred.groupby('customer_ID')['D_predicted'].last()
# D_predicted_mode=D_pred.groupby('customer_ID')['D_predicted'].apply(lambda x: x.mode().iloc[0])
D_predicted_mode=D_predicted_mode.to_frame()
D_predicted_mode.reset_index(level=0, inplace=True)

### Prediction of A & F

In [81]:
#MODEL FOR A

#splitting my train data in to train & validation
y = my_train['A_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 293)
validation data size (133050, 293)
             precision    recall  f1-score   support

          0       0.84      0.78      0.81     28858
          1       0.82      0.94      0.88     81778
          2       0.84      0.47      0.60     22414

avg / total       0.83      0.83      0.82    133050

0.8287561067267945


In [82]:
#Applying the model on test Data
A_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_A=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
A_pred['A_predicted']=logreg.predict(my_test_A)

A_predicted_mode=A_pred.groupby('customer_ID')['A_predicted'].last()
# A_predicted_mode=A_pred.groupby('customer_ID')['A_predicted'].apply(lambda x: x.mode().iloc[0])
A_predicted_mode=A_predicted_mode.to_frame()
A_predicted_mode.reset_index(level=0, inplace=True)

In [83]:
#MODEL FOR F

#splitting my train data in to train & validation
y = my_train['F_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 294)
validation data size (133050, 294)
             precision    recall  f1-score   support

          0       0.90      0.92      0.91     42029
          1       0.80      0.79      0.80     33326
          2       0.76      0.84      0.80     49919
          3       0.79      0.15      0.25      7776

avg / total       0.82      0.82      0.80    133050

0.8154077414505825


In [84]:
#Applying the model on test Data
my_test_F=my_test.copy()
F_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_F=my_test_F.merge(A_predicted_mode, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_F=my_test_F.drop(['record_type','customer_ID'], axis=1)

In [85]:
logreg = LogisticRegression()
logreg.fit(X, y)
F_pred['F_predicted']=logreg.predict(my_test_F)

F_predicted_mode=F_pred.groupby('customer_ID')['F_predicted'].last()
# F_predicted_mode=F_pred.groupby('customer_ID')['F_predicted'].apply(lambda x: x.mode().iloc[0])
F_predicted_mode=F_predicted_mode.to_frame()
F_predicted_mode.reset_index(level=0, inplace=True)

### Prediction of B & E

In [86]:
#MODEL FOR B

#splitting my train data in to train & validation
y = my_train['B_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 293)
validation data size (133050, 293)
             precision    recall  f1-score   support

          0       0.88      0.92      0.90     69564
          1       0.90      0.86      0.88     63486

avg / total       0.89      0.89      0.89    133050

0.889793310785419


In [87]:
#Applying the model on test Data
B_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_B=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
B_pred['B_predicted']=logreg.predict(my_test_B)

B_predicted_mode=B_pred.groupby('customer_ID')['B_predicted'].last()
# B_predicted_mode=B_pred.groupby('customer_ID')['B_predicted'].apply(lambda x: x.mode().iloc[0])
B_predicted_mode=B_predicted_mode.to_frame()
B_predicted_mode.reset_index(level=0, inplace=True)

In [88]:
#MODEL FOR E

#splitting my train data in to train & validation
y = my_train['E_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (532199, 294)
validation data size (133050, 294)
             precision    recall  f1-score   support

          0       0.92      0.92      0.92     71032
          1       0.91      0.90      0.90     62018

avg / total       0.91      0.91      0.91    133050

0.9112438932732055


In [89]:
#Applying the model on test Data
my_test_E=my_test.copy()
E_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_E=my_test_E.merge(B_predicted_mode, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_E=my_test_E.drop(['record_type','customer_ID'], axis=1)

In [90]:
logreg = LogisticRegression()
logreg.fit(X, y)
E_pred['E_predicted']=logreg.predict(my_test_E)
E_predicted_mode=E_pred.groupby('customer_ID')['E_predicted'].last()
# E_predicted_mode=E_pred.groupby('customer_ID')['E_predicted'].apply(lambda x: x.mode().iloc[0])
E_predicted_mode=E_predicted_mode.to_frame()
E_predicted_mode.reset_index(level=0, inplace=True)

In [91]:
F_predicted_mode.F_predicted.unique()

array([2, 1, 0, 3], dtype=int64)

In [92]:
final_predictions_df=A_predicted_mode.merge(B_predicted_mode,on='customer_ID').merge(C_predicted_mode,on='customer_ID').merge(D_predicted_mode,on='customer_ID').merge(E_predicted_mode,on='customer_ID').merge(F_predicted_mode,on='customer_ID').merge(G_predicted_mode,on='customer_ID')


In [93]:
final_predictions_df.head()

customer_ID  A_predicted  B_predicted  C_predicted  D_predicted  \
0     10000001            2            1            1            3   
1     10000002            1            0            2            3   
2     10000003            1            0            3            2   
3     10000004            2            0            1            1   
4     10000006            1            0            1            1   

   E_predicted  F_predicted  G_predicted  
0            1            2            2  
1            1            1            2  
2            0            2            2  
3            1            2            2  
4            0            0            1

In [94]:
for C in final_predictions_df.columns:
    final_predictions_df[C]=final_predictions_df[C].astype(str)
final_predictions_df['plan']=final_predictions_df['A_predicted'] + final_predictions_df['B_predicted'] + final_predictions_df['C_predicted'] + final_predictions_df['D_predicted'] + final_predictions_df['E_predicted'] + final_predictions_df['F_predicted'] + final_predictions_df['G_predicted']

final_predictions_df=final_predictions_df[['customer_ID','plan']]
final_predictions_df.to_csv(r'C:/Users/karth/Documents/Kaggle/Purchase Prediction/First_model.csv', index=False)

### Using only the Last row and Modelling